In [ ]:
%matplotlib inline
import os
import shutil
import flopy
import pandas as pd
import pyemu
import helpers

In [ ]:
# safe_org_d = os.path.join("..", "models", "synthetic-valley-working-annual")
# safe_org_d = os.path.join("..", "models", "synthetic-valley-working-monthly")
safe_org_d = os.path.join("..", "models", "synthetic-valley-advanced-monthly")
# safe_org_d = os.path.join("..", "models", "synthetic-valley-advanced-annual")
assert os.path.exists(safe_org_d)

In [ ]:
tmp_d = "temp"
if os.path.exists(tmp_d):
    shutil.rmtree(tmp_d)

In [ ]:
sim = flopy.mf6.MFSimulation.load(sim_ws=safe_org_d)
sim.set_sim_path(tmp_d)
gwf = sim.get_model()
gwf.set_all_data_external(external_data_folder=".")
sim.write_simulation()

In [ ]:
sim.run_simulation()

In [ ]:
import importlib

importlib.reload(helpers)
helpers.process_csv_files(model_ws=tmp_d)

In [ ]:
pf = pyemu.utils.PstFrom(
    tmp_d,
    "model_and_pest_files",
    remove_existing=True,
    spatial_reference=gwf.modelgrid,
    zero_based=False,
    start_datetime=gwf.start_datetime,
    echo=False,
    chunk_len=1000000,
)

In [ ]:
pf.add_py_function("helpers.py", "process_csv_files()", is_pre_cmd=False)

In [ ]:
pf.mod_sys_cmds.append("mf6")

In [ ]:
obs_csv_files = [
    f for f in os.listdir(pf.new_d) if f.startswith("sv.gwf") and f.endswith(".csv")
]
obs_csv_files.extend(
    [f for f in os.listdir(pf.new_d) if f.startswith("sv.lake") and f.endswith(".csv")]
)
obs_csv_files.extend(
    [f for f in os.listdir(pf.new_d) if f.startswith("sv.riv") and f.endswith(".csv")]
)
obs_csv_files.extend(
    [f for f in os.listdir(pf.new_d) if f.startswith("sv.sfr") and f.endswith(".csv")]
)


obs_csv_files

In [ ]:
for obs_csv_file in obs_csv_files:
    print(obs_csv_file)
    prefix = obs_csv_file.replace(".", "-")
    df = pd.read_csv(os.path.join(pf.new_d, obs_csv_file), index_col=0)
    pf.add_observations(
        obs_csv_file,
        index_cols="datetime",
        use_cols=df.columns.to_list(),
        prefix=prefix,
        obsgp=prefix,
        ofile_sep=",",
    )

In [ ]:
k_files = [f for f in os.listdir(pf.new_d) if f.startswith("sv.npf_k_layer")]
assert len(k_files) == gwf.dis.nlay.data

In [ ]:
k_files.sort()
k_files

In [ ]:
pp_v = pyemu.geostats.ExpVario(contribution=1.0, a=5000)
pp_geostruct = pyemu.geostats.GeoStruct(variograms=pp_v, transform="log")

In [ ]:
df = pf.add_parameters(
    k_files[:2],
    par_type="pilotpoints",
    pp_options={"pp_space": 3},
    lower_bound=0.1,
    upper_bound=10.0,
    geostruct=pp_geostruct,
    par_name_base="hk-pp-wt",
    pargp="hk-pp-wt",
)

In [ ]:
df.head()

In [ ]:
df = pf.add_parameters(
    k_files[:2],
    par_type="constant",
    lower_bound=0.5,
    upper_bound=2.0,
    par_name_base="hk-cn-wt",
    pargp="hk-cn-wt",
)
df

In [ ]:
df = pf.add_parameters(
    k_files[2],
    par_type="pilotpoints",
    pp_options={"pp_space": 3},
    lower_bound=0.1,
    upper_bound=10.0,
    geostruct=pp_geostruct,
    par_name_base="hk-pp-conf",
    pargp="hk-pp-conf",
)
df = pf.add_parameters(
    k_files[2],
    par_type="constant",
    lower_bound=0.5,
    upper_bound=2.0,
    par_name_base="hk-cn-conf",
    pargp="hk-cn-conf",
)

In [ ]:
df = pf.add_parameters(
    k_files[3:],
    par_type="pilotpoints",
    pp_options={"pp_space": 3},
    lower_bound=0.1,
    upper_bound=10.0,
    geostruct=pp_geostruct,
    par_name_base="hk-pp-aq",
    pargp="hk-pp-aq",
)
df = pf.add_parameters(
    k_files[3:],
    par_type="constant",
    lower_bound=0.5,
    upper_bound=2.0,
    par_name_base="hk-cn-aq",
    pargp="hk-cn-aq",
)

In [ ]:
for k_file in k_files[::2]:
    print(k_file)
    pf.add_observations(
        k_file,
        obsgp=k_file.split(".")[1].replace("_", "-"),
        prefix=k_file.split(".")[1].replace("_", "-"),
    )

In [ ]:
ss_files = [f for f in os.listdir(pf.new_d) if f.startswith("sv.sto_ss")]
assert len(ss_files) == gwf.dis.nlay.data

In [ ]:
df = pf.add_parameters(
    ss_files[:2],
    par_type="pilotpoints",
    pp_options={"pp_space": 3},
    lower_bound=0.2,
    upper_bound=5.0,
    geostruct=pp_geostruct,
    par_name_base="ss-pp-wt",
    pargp="ss-pp-wt",
)
df = pf.add_parameters(
    ss_files[:2],
    par_type="constant",
    lower_bound=0.5,
    upper_bound=2.0,
    par_name_base="ss-cn-wt",
    pargp="ss-cn-wt",
)
df = pf.add_parameters(
    ss_files[2],
    par_type="pilotpoints",
    pp_options={"pp_space": 3},
    lower_bound=0.2,
    upper_bound=5.0,
    geostruct=pp_geostruct,
    par_name_base="ss-pp-conf",
    pargp="ss-pp-conf",
)
df = pf.add_parameters(
    ss_files[2],
    par_type="constant",
    lower_bound=0.5,
    upper_bound=2.0,
    par_name_base="ss-cn-conf",
    pargp="ss-cn-conf",
)
df = pf.add_parameters(
    ss_files[3:],
    par_type="pilotpoints",
    pp_options={"pp_space": 3},
    lower_bound=0.2,
    upper_bound=5.0,
    geostruct=pp_geostruct,
    par_name_base="ss-pp-aq",
    pargp="ss-pp-aq",
)
df = pf.add_parameters(
    ss_files[3:],
    par_type="constant",
    lower_bound=0.5,
    upper_bound=2.0,
    par_name_base="ss-cn-aq",
    pargp="ss-cn-aq",
)

In [ ]:
sy_files = [f for f in os.listdir(pf.new_d) if f.startswith("sv.sto_sy")]
assert len(sy_files) == gwf.dis.nlay.data
sy_files.sort()
sy_file = sy_files[0]
assert "layer1" in sy_file

In [ ]:
df = pf.add_parameters(
    sy_file,
    par_type="pilotpoints",
    pp_options={"pp_space": 3},
    lower_bound=0.2,
    upper_bound=5.0,
    geostruct=pp_geostruct,
    par_name_base="sy-pp-wt",
    pargp="sy-pp-wt",
)
df = pf.add_parameters(
    sy_file,
    par_type="constant",
    lower_bound=0.5,
    upper_bound=2.0,
    par_name_base="sy-cn-wt",
    pargp="sy-cn-wt",
)

In [ ]:
rech_files = [
    f for f in os.listdir(pf.new_d) if f.startswith("sv.rch_stress_period_data_")
]
if len(rech_files) == 0:
    uzf_pak_file = "sv.uzf_packagedata.txt"
    if os.path.exists(os.path.join(pf.new_d, uzf_pak_file)):
        pass
    else:
        raise Exception("didnt find any rech files or uzg package data file")

else:
    assert len(rech_files) == sim.tdis.nper.data
    rech_files.sort()
    df = pf.add_parameters(
        rech_files,
        par_type="constant",
        lower_bound=0.8,
        upper_bound=1.2,
        par_name_base="rech-cn",
        pargp="rech-cn",
        transform=None,
        index_cols=[0, 1, 2],
        use_cols=[3],
        mfile_skip=0,
    )
    for rech_file in rech_files:
        kper = int(rech_file.split(".")[1].split("_")[-1]) - 1
        df = pf.add_parameters(
            rech_file,
            par_type="constant",
            lower_bound=0.8,
            upper_bound=1.2,
            par_name_base="rech-tcn_kper:{0}".format(kper),
            pargp="rech-tcn_kper:{0}".format(kper),
            transform=None,
            index_cols=[0, 1, 2],
            use_cols=[3],
            mfile_skip=0,
        )

In [ ]:
predwel_files = [
    f
    for f in os.listdir(pf.new_d)
    if f.startswith("sv.prediction.well_stress_period_data_")
]
assert len(predwel_files) > 0

In [ ]:
for pwel_file in predwel_files:
    kper = kper = int(pwel_file.split(".")[2].split("_")[-1]) - 1
    pf.add_parameters(
        pwel_file,
        par_type="grid",
        index_cols=[0, 1, 2],
        use_cols=[3],
        upper_bound=0,
        lower_bound=-500000,
        pargp="predwel_kper:{0}".format(kper),
        par_name_base="predwel_kper:{0}".format(kper),
        transform="none",
        initial_value=-30000,
        par_style="direct",
    )

In [ ]:
pf.build_pst(filename="pest.pst")

In [ ]:
pst = pf.pst
pst.control_data.noptmax = 0
pst.write(os.path.join(pf.new_d, "pest.pst"), version=2)

In [ ]:
pyemu.os_utils.run("pestpp-ies pest.pst", cwd=pf.new_d)

In [ ]:
pst.set_res(os.path.join(pf.new_d, "pest.base.rei"))
pst.phi

In [ ]:
par = pst.parameter_data

predpar = par.loc[par.pname == "predwel", :]
assert len(predpar) > 0
par.loc[predpar.parnme, "partrans"] = "fixed"

In [ ]:
pe = pf.draw(num_reals=1000)

In [ ]:
pe.enforce()
pe.to_csv(os.path.join(pf.new_d, "prior.csv"))
pst.pestpp_options["ies_par_en"] = "prior.csv"

In [ ]:
pst.parameter_data.loc[pe.columns, "parval1"] = pe.iloc[0, :].values
pst.control_data.noptmax = 0
pst.write(os.path.join(pf.new_d, "test.pst"), version=2)
pyemu.os_utils.run("pestpp-ies test.pst", cwd=pf.new_d)

In [ ]:
pst.pestpp_options["ies_num_reals"] = 30
pst.control_data.noptmax = -1

In [ ]:
pst.write(os.path.join(pf.new_d, "pest.pst"), version=2)

In [ ]:
pyemu.os_utils.start_workers(
    pf.new_d,
    "pestpp-ies",
    "pest.pst",
    worker_root=".",
    num_workers=10,
    master_dir="master_prior_mc",
)